### Usually data is loaded from disk, here we'll use a Tensor using tf.data

In [1]:
import tensorflow as tf

X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [9]:
@tf.function
def iterate_dataset(ds):
    for item in ds:
        tf.print(item)  # Use tf.print() for TensorFlow graphs

iterate_dataset(dataset)

0
1
2
3
4
5
6
7
8
9


In [11]:
# Keeps nested structure
X_nested = {"a": ([1,2,3], [4,5,6]), "b": [7,8,9]}
dataset = tf.data.Dataset.from_tensor_slices(X_nested)
for item in dataset:
    print(item)

{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=4>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=7>}
{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=int32, numpy=5>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=8>}
{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=3>, <tf.Tensor: shape=(), dtype=int32, numpy=6>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=9>}


In [12]:
# Chaining transformations, batch with `drop_remainder=True` will remove the last
dataset = tf.data.Dataset.from_tensor_slices(tf.range(10))
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [13]:
dataset = dataset.map(lambda x: x * 2)
for item in dataset:
    print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
tf.Tensor([16 18], shape=(2,), dtype=int32)


In [15]:
dataset = dataset.filter(lambda x: tf.reduce_sum(x) > 50)
for item in dataset:
    tf.print(item)

[14 16 18 ... 2 4 6]
[8 10 12 ... 16 18 0]
[2 4 6 ... 10 12 14]


In [16]:
for item in dataset.take(2):
    print(item)

tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)


Shuffling:

In [17]:
dataset = tf.data.Dataset.range(10).repeat(2)
dataset = dataset.shuffle(buffer_size=4, seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([1 4 2 3 5 0 6], shape=(7,), dtype=int64)
tf.Tensor([9 8 2 0 3 1 4], shape=(7,), dtype=int64)
tf.Tensor([5 7 9 6 7 8], shape=(6,), dtype=int64)


Interleaving lines from multiple files

In [21]:
train_filepaths = ['./data/housing/housing.csv', './data/housing/housing_train_01.csv']

`list_files()` shuffles filepaths

In [22]:
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)

Skipping the header and taking 5 TextLineDataset

In [24]:
n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers)

In [25]:
for line in dataset.take(5):
    print(line)

tf.Tensor(b'-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY', shape=(), dtype=string)
tf.Tensor(b'-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY', shape=(), dtype=string)
tf.Tensor(b'-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY', shape=(), dtype=string)
tf.Tensor(b'-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY', shape=(), dtype=string)
tf.Tensor(b'-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY', shape=(), dtype=string)


preprocessing

In [26]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Generate sample training data (1000 rows, 8 features)
np.random.seed(42)
X_train_sample = np.random.rand(1000, 8) * 10  # Random values between 0 and 10

# Compute mean and std using StandardScaler
scaler = StandardScaler()
scaler.fit(X_train_sample)  # Compute mean and std

X_mean = scaler.mean_  # Mean of each feature
X_std = scaler.scale_  # Standard deviation of each feature

n_inputs = 8

def parse_csv_line(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    return tf.stack(fields[:-1]), tf.stack(fields[-1:])

def preprocess(line):
    x, y = parse_csv_line(line)
    return (x - X_mean) / X_std, y

In [27]:
preprocess(b'4.2083,44.0,5.3232,0.9171,846.0,2.3370,37.47,-122.2,2.782')

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([-2.8835964e-01,  1.3298166e+01,  1.8436758e-01, -1.4383570e+00,
         2.9296912e+02, -9.6995091e-01,  1.1021225e+01, -4.3763279e+01],
       dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.782], dtype=float32)>)

everything together: `prefetch()` for performance, `num_parallel_calls` multiple CPU cores

In [29]:
def csv_reader_dataset(filepaths, n_readers=5, n_read_threads=None,
                       n_parse_threads=5, shuffle_buffer_size=10_000, seed=42,
                       batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths, seed=seed)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.shuffle(shuffle_buffer_size, seed=seed)
    return dataset.batch(batch_size).prefetch(1) # works in parallel getting the next batch ready while processing

TF function that trains model for whole epoch (speeds up training)

In [33]:
n_epochs = 5

@tf.function
def train_one_epoch(model, optimizer, loss_fn, train_set):
    for X_batch, y_batch in train_set:
        with tf.GradientTape() as tape:
            y_pred = model(X_batch)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
loss_fn = tf.keras.losses.MeanSquaredError()
for epoch in range(n_epochs):
    print("\rEpoch {}/{}".format(epoch + 1, n_epochs), end="")
    train_one_epoch(model, optimizer, loss_fn, train_set)

Epoch 5/5

### TFRecord format is TensorFlow's preferred format

holds sequence of binary records, and CRC checksum

In [34]:
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write(b"This is the first record")
    f.write(b"...and this is the second record")

In [35]:
filepaths = ["my_data.tfrecord"]
dataset = tf.data.TFRecordDataset(filepaths)
for item in dataset:
    print(item)

tf.Tensor(b'This is the first record', shape=(), dtype=string)
tf.Tensor(b'...and this is the second record', shape=(), dtype=string)


2025-02-09 08:49:51.669677: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:370] TFRecordDataset `buffer_size` is unspecified, default to 262144


Instead of writing raw bytes, use TensorFlow's tf.train.Example format to store structured data.

In [36]:
# Create a TFRecord file with structured data
with tf.io.TFRecordWriter("my_data2.tfrecord") as writer:
    for text in [b"This is the first record", b"...and this is the second record"]:
        feature = {
            "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[text]))
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())  # Serialize and write

# Read TFRecord file
def parse_example(example_proto):
    feature_description = {
        "text": tf.io.FixedLenFeature([], tf.string)  # Define expected structure
    }
    parsed_example = tf.io.parse_single_example(example_proto, feature_description)
    return parsed_example["text"]

dataset = tf.data.TFRecordDataset(["my_data2.tfrecord"])
dataset = dataset.map(parse_example)  # Decode each record

# Print the processed dataset
for item in dataset:
    print(item.numpy().decode("utf-8"))  # Convert back to a string

This is the first record
...and this is the second record


Compress Records

In [38]:
options = tf.io.TFRecordOptions(compression_type="GZIP")
with tf.io.TFRecordWriter("my_compressed.tfrecord", options) as f:
    f.write(b"Compress, compressing, compressed!")

In [39]:
dataset = tf.data.TFRecordDataset(["my_compressed.tfrecord"], compression_type="GZIP")

### protobuf

In [42]:
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Feature, Features, Example

person_example = Example(
    features=Features(
        feature={
            "name": Feature(bytes_list=BytesList(value=[b"Mathias"])),
            "id": Feature(int64_list=Int64List(value=[420])),
            "emails": Feature(bytes_list=BytesList(value=[b"a@b.com", b"c@d.com"]))
        }))

^ this could be a helper function, as its a bit verbose

In [43]:
with tf.io.TFRecordWriter("my_contacts.tfrecord") as f:
    for _ in range(5):
        f.write(person_example.SerializeToString())

In [44]:
feature_description = {
    "name": tf.io.FixedLenFeature([], tf.string, default_value=""),
    "id": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "emails": tf.io.VarLenFeature(tf.string),
}

def parse(serialized_example):
    return tf.io.parse_single_example(serialized_example, feature_description)

dataset = tf.data.TFRecordDataset(["my_contacts.tfrecord"]).map(parse)
for parsed_example in dataset:
    print(parsed_example)

{'emails': SparseTensor(indices=tf.Tensor(
[[0]
 [1]], shape=(2, 1), dtype=int64), values=tf.Tensor([b'a@b.com' b'c@d.com'], shape=(2,), dtype=string), dense_shape=tf.Tensor([2], shape=(1,), dtype=int64)), 'id': <tf.Tensor: shape=(), dtype=int64, numpy=420>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'Mathias'>}
{'emails': SparseTensor(indices=tf.Tensor(
[[0]
 [1]], shape=(2, 1), dtype=int64), values=tf.Tensor([b'a@b.com' b'c@d.com'], shape=(2,), dtype=string), dense_shape=tf.Tensor([2], shape=(1,), dtype=int64)), 'id': <tf.Tensor: shape=(), dtype=int64, numpy=420>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'Mathias'>}
{'emails': SparseTensor(indices=tf.Tensor(
[[0]
 [1]], shape=(2, 1), dtype=int64), values=tf.Tensor([b'a@b.com' b'c@d.com'], shape=(2,), dtype=string), dense_shape=tf.Tensor([2], shape=(1,), dtype=int64)), 'id': <tf.Tensor: shape=(), dtype=int64, numpy=420>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'Mathias'>}
{'emails': SparseTensor(indices=tf

In [45]:
tf.sparse.to_dense(parsed_example["emails"], default_value=b"")

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'a@b.com', b'c@d.com'], dtype=object)>

In [46]:
parsed_example["emails"].values

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'a@b.com', b'c@d.com'], dtype=object)>

Batch parse:

In [47]:
def parse(serialized_examples):
    return tf.io.parse_example(serialized_examples, feature_description)

dataset = tf.data.TFRecordDataset(["my_contacts.tfrecord"]).batch(2).map(parse)
for parsed_examples in dataset:
    print(parsed_examples)

{'emails': SparseTensor(indices=tf.Tensor(
[[0 0]
 [0 1]
 [1 0]
 [1 1]], shape=(4, 2), dtype=int64), values=tf.Tensor([b'a@b.com' b'c@d.com' b'a@b.com' b'c@d.com'], shape=(4,), dtype=string), dense_shape=tf.Tensor([2 2], shape=(2,), dtype=int64)), 'id': <tf.Tensor: shape=(2,), dtype=int64, numpy=array([420, 420])>, 'name': <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'Mathias', b'Mathias'], dtype=object)>}
{'emails': SparseTensor(indices=tf.Tensor(
[[0 0]
 [0 1]
 [1 0]
 [1 1]], shape=(4, 2), dtype=int64), values=tf.Tensor([b'a@b.com' b'c@d.com' b'a@b.com' b'c@d.com'], shape=(4,), dtype=string), dense_shape=tf.Tensor([2 2], shape=(2,), dtype=int64)), 'id': <tf.Tensor: shape=(2,), dtype=int64, numpy=array([420, 420])>, 'name': <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'Mathias', b'Mathias'], dtype=object)>}
{'emails': SparseTensor(indices=tf.Tensor(
[[0 0]
 [0 1]], shape=(2, 2), dtype=int64), values=tf.Tensor([b'a@b.com' b'c@d.com'], shape=(2,), dtype=string), dense_shap

2025-02-09 09:26:29.261646: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Its best to have any preprocessing layers directly in your model, so its easily portable to production and it can preprocess data on the fly

measure feature means and variances before training

This eliminates the risk of preprocessing mismatch

In [49]:
norm_layer = tf.keras.layers.Normalization()
model = tf.keras.models.Sequential([
    norm_layer,
    tf.keras.layers.Dense(1)
])
model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(learning_rate=2e-3))
# norm_layer.adapt(X_train)
# model.fit(X_train, y_trian, validation_data=(X_valid, y_valid), epochs=5)

normalizing the whole training set just once before training

In [51]:
norm_layer = tf.keras.layers.Normalization()
# norm_layer.adapt(X_train)
# X_train_scaled = norm_layer(X_train)
# X_valid_scaled = norm_layer(X_valid)

train model on scaled data without Normalization:

In [54]:
model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(learning_rate=2e-3))
# model.fit(X_train_scaled, y_train, epochs=5, validation_data=(X_valid_scaled, y_valid))

we need to prepocess the inputs when deploying to production:

this will take care of both preprocessing its inputs and making predictions

In [56]:
final_model = tf.keras.Sequential([norm_layer, model])
# X_new = X_test[:3]
# y_pred = final_model(X_new)

adapted normalization layer to input features of each batch in dataset

In [58]:
# dataset = dataset.map(lambda X, y: (norm_layer(X), y))

Custom layer, if more features needed than Keras preprocessing layers

In [60]:
import numpy as np

class MyNormalization(tf.keras.layers.Layer):
    def adapt(self, X):
        self.mean_ = np.mean(X, axis=0, keepdims=True)
        self.std_ = np.std(X, axis=0, keepdims=True)

    def call(self, inputs):
        eps = tf.keras.backend.epsilon()
        return (inputs - self.mean_) / (self.std_ + eps)

### Discretization layer

numreical features into categorical features by mapping value ranges (bins) to categories

less than 18, 18 to 50 (not included), 50 and over:

In [61]:
age = tf.constant([[10.], [93.], [57.], [18.], [37.], [5.]])
discretize_layer = tf.keras.layers.Discretization(bin_boundaries=[18., 50.])
age_categories = discretize_layer(age)
age_categories

<tf.Tensor: shape=(6, 1), dtype=int64, numpy=
array([[0],
       [2],
       [2],
       [1],
       [1],
       [0]])>

we can do it based on percentiles `num_bins=3` values below the 33rd and 66th percentiles (10 and 37)

In [62]:
discretize_layer = tf.keras.layers.Discretization(num_bins=3)
discretize_layer.adapt(age)
age_categories = discretize_layer(age)
age_categories

<tf.Tensor: shape=(6, 1), dtype=int64, numpy=
array([[1],
       [2],
       [2],
       [1],
       [2],
       [0]])>

should not be used for neural network, we can use one-hot encoding

In [63]:
onehot_layer = tf.keras.layers.CategoryEncoding(num_tokens=3)
onehot_layer(age_categories)

<tf.Tensor: shape=(6, 3), dtype=float32, numpy=
array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)>

encoding more than one categorical feature at a time

In [64]:
two_age_categories = np.array([[1, 0], [2, 2], [2, 0]])
onehot_layer(two_age_categories)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1., 1., 0.],
       [0., 0., 1.],
       [1., 0., 1.]], dtype=float32)>

`output_mode="count"` how many times each category occured

Multi-hot encoding lose information both [0, 1] amd [1, 0] are encoded as [1, 1, 0] we can avoid this by one-hot encode each feature separately and concat the outputs

In [67]:
onehot_layer = tf.keras.layers.CategoryEncoding(num_tokens=3 + 3)
onehot_layer(two_age_categories + [0, 3]) # adds 3 to the second feature

<tf.Tensor: shape=(3, 6), dtype=float32, numpy=
array([[0., 1., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 1.],
       [0., 0., 1., 1., 0., 0.]], dtype=float32)>

^ first three columns correspond to the first feature, and the last three correspond to the second feature

In [70]:
cities = ["Auckland", "Paris", "Paris", "Santa Monica"]
str_lookup_layer = tf.keras.layers.StringLookup()
str_lookup_layer.adapt(cities)
str_lookup_layer([["Paris"], ["Auckland"], ["Auckland"], ["Montreal"]])

<tf.Tensor: shape=(4, 1), dtype=int64, numpy=
array([[1],
       [3],
       [3],
       [0]])>

one-hot vector instead of an integer:

In [71]:
str_lookup_layer = tf.keras.layers.StringLookup(output_mode="one_hot")
str_lookup_layer.adapt(cities)
str_lookup_layer([["Paris"], ["Auckland"], ["Auckland"], ["Montreal"]])

<tf.Tensor: shape=(4, 4), dtype=int64, numpy=
array([[0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0]])>

out-of-vocabulary (OOV) buckets: each unknown category will get mapped pseudorandomly to one OOV buckets, using a hash function modulo the number of OOV buckets. This will allow the model to distinguish at least some of the rare categories.

In [73]:
str_lookup_layer = tf.keras.layers.StringLookup(num_oov_indices=5)
str_lookup_layer.adapt(cities)
str_lookup_layer([["Paris"], ["Auckland"], ["Foo"], ["Bar"], ["Baz"]])

<tf.Tensor: shape=(5, 1), dtype=int64, numpy=
array([[5],
       [7],
       [4],
       [3],
       [4]])>

### Hashing layer

In [74]:
hashing_layer = tf.keras.layers.Hashing(num_bins=10)
hashing_layer([["Paris"], ["Tokyo"], ["Auckland"], ["Montreal"]])

<tf.Tensor: shape=(4, 1), dtype=int64, numpy=
array([[0],
       [1],
       [9],
       [1]])>

### Embeddings

encoding some categories:

In [75]:
tf.random.set_seed(42)
embedding_layer = tf.keras.layers.Embedding(input_dim=5, output_dim=2)
embedding_layer(np.array([2, 4, 2]))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.0215165 ,  0.03553423],
       [-0.02882874, -0.0356148 ],
       [-0.0215165 ,  0.03553423]], dtype=float32)>

Embedding layer is initialized randomly

embedding categorical text attribute, chain StringLookup layer and Embedding layer

In [76]:
tf.random.set_seed(42)
ocean_prox = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"]
str_lookup_layer = tf.keras.layers.StringLookup()
str_lookup_layer.adapt(ocean_prox)
lookup_and_embed = tf.keras.Sequential([
    str_lookup_layer,
    tf.keras.layers.Embedding(input_dim=str_lookup_layer.vocabulary_size(), output_dim=2)
])

In [78]:
lookup_and_embed(np.array([["<1H OCEAN"], ["ISLAND"], ["<1H OCEAN"]]))

<tf.Tensor: shape=(3, 1, 2), dtype=float32, numpy=
array([[[-0.00636964, -0.0307073 ]],

       [[ 0.02456259,  0.01703635]],

       [[-0.00636964, -0.0307073 ]]], dtype=float32)>

### Text preprocessing

In [80]:
train_data = ["To be", "!(to be)", "That's the question", "Be, be, be."]
text_vec_layer = tf.keras.layers.TextVectorization()
text_vec_layer.adapt(train_data)
text_vec_layer(["Be good!", "Question: be or be?"])

<tf.Tensor: shape=(2, 4), dtype=int64, numpy=
array([[2, 1, 0, 0],
       [6, 2, 1, 2]])>

^ since first sentence is sort it is padded with 0's

preferable to set "tf_idf", which stands for term-frequency × inverse-document-frequency (TF-IDF). This is similar to the count encoding, but words that occur frequently in the training data are downweighted, rare words are upweighted. 

In [83]:
text_vec_layer = tf.keras.layers.TextVectorization(output_mode="tf_idf")
text_vec_layer.adapt(train_data)
text_vec_layer(["Be good!", "Question: be or be?"])

<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
array([[0.96725637, 0.6931472 , 0.        , 0.        , 0.        ,
        0.        ],
       [0.96725637, 1.3862944 , 0.        , 0.        , 0.        ,
        1.0986123 ]], dtype=float32)>

weight equal to `log(1 + d / (f + 1))`

d: total number of sentences

f: counts how many of training sentences contain given word

Since the word "be" occurs twice in the sentence "Question: be or be?", it gets encoded as 2 × log(1 + 4 / (1 + 3)) ≈ 1.3862944.

### Tensorflow hub (not sure if people are using this over HuggingFace?)

In [85]:
!pip install tensorflow_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


sentence encoder

In [88]:
import tensorflow_hub as hub

# pre-trained on Google News 7B corpus
hub_layer = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim50/2")
sentence_embeddings = hub_layer(tf.constant(["To be", "Not to be"]))
sentence_embeddings.numpy().round(2)

array([[-0.25,  0.28,  0.01,  0.1 ,  0.14,  0.16,  0.25,  0.02,  0.07,
         0.13, -0.19,  0.06, -0.04, -0.07,  0.  , -0.08, -0.14, -0.16,
         0.02, -0.24,  0.16, -0.16, -0.03,  0.03, -0.14,  0.03, -0.09,
        -0.04, -0.14, -0.19,  0.07,  0.15,  0.18, -0.23, -0.07, -0.08,
         0.01, -0.01,  0.09,  0.14, -0.03,  0.03,  0.08,  0.1 , -0.01,
        -0.03, -0.07, -0.1 ,  0.05,  0.31],
       [-0.2 ,  0.2 , -0.08,  0.02,  0.19,  0.05,  0.22, -0.09,  0.02,
         0.19, -0.02, -0.14, -0.2 , -0.04,  0.01, -0.07, -0.22, -0.1 ,
         0.16, -0.44,  0.31, -0.1 ,  0.23,  0.15, -0.05,  0.15, -0.13,
        -0.04, -0.08, -0.16, -0.1 ,  0.13,  0.13, -0.18, -0.04,  0.03,
        -0.1 , -0.07,  0.07,  0.03, -0.08,  0.02,  0.05,  0.07, -0.14,
        -0.1 , -0.18, -0.13, -0.04,  0.15]], dtype=float32)

### Images preprocessing

`tf.keras.layers.Rescaling(scale=2/255, offset=-1)` scaled the values from 0 -> 255 to -1 -> 1

In [90]:
from sklearn.datasets import load_sample_images

images = load_sample_images()["images"]
crop_image_layer = tf.keras.layers.CenterCrop(height=100, width=100)
crapped_images = crop_image_layer(images)

preprocessing layers are based on TF low-level API most of these methods in this notebook call a different method for example: `Normalization` -> `tf.nn.moments()`, `Discretization` -> `tf.raw_ops.Bucketize()` etc

In [91]:
!pip install tensorflow_datasets

  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 34.6 MB/s eta 0:00:00
Using cached click-8.1.8-py3-none-any.whl (98 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 66.7 MB/s eta 0:00:0000:0100:01
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Created wheel for dm-tree: filename=dm_tree-0.1.9-cp312-cp312-macosx_15_0_arm64.whl size=112050 sha256=c65c29d23e5b346f25a99bdfc903eaabdeda5c1c2bcb78ee9f1db8ea8a413582
  Stored in directory: /Users/mathias/Library/Caches/pip/wheels/89/01/af/d33b8877c63c2b90659a0eba4cffefbd049665c311d0a3d23a
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21548 sha256=7b578e5f1ccdab9128ed3326f3e45a0da0c795ceb60d7e6daed21ac7af18bfb4
  Stored in directory: /Users/mathias/Library/Caches/pip/wheels/e7/e6/28/864bdfee5339

In [92]:
import tensorflow_datasets as tfds

datasets = tfds.load(name="mnist")
mnist_train, mnist_test = datasets["train"], datasets["test"]

2025-02-09 19:39:34.961788: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /Users/mathias/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [93]:
for batch in mnist_train.shuffle(10_000, seed=42).batch(32).prefetch(1):
    images = batch["image"]
    labels = batch["label"]

2025-02-09 19:40:35.985234: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


Keras expects each item to be a tuple

In [94]:
mnist_train = mnist_train.shuffle(buffer_size=10_000, seed=42).batch(32)
mnist_train = mnist_train.map(lambda items: (items["image"], items["label"]))
mnist_train = mnist_train.prefetch(1)

In [97]:
train_set, valid_set, test_set = tfds.load(
    name="mnist",
    split=["train[:90%]", "train[90%:]", "test"],
    as_supervised=True
)
train_set = train_set.shuffle(buffer_size=10_000, seed=42).batch(32).prefetch(1)
valid_set = valid_set.batch(32).cache()
test_set = test_set.batch(32).cache()
tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=5)
test_loss, test_accuracy = model.evaluate(test_set)

Epoch 1/5


/Users/mathias/.pyenv/versions/3.12.0/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1688/1688 ━━━━━━━━━━━━━━━━━━━━ 2s 699us/step - accuracy: 0.7631 - loss: 18.5642 - val_accuracy: 0.8822 - val_loss: 6.0332
Epoch 2/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 1s 537us/step - accuracy: 0.8783 - loss: 5.6927 - val_accuracy: 0.8788 - val_loss: 5.6601
Epoch 3/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 1s 539us/step - accuracy: 0.8843 - loss: 5.2441 - val_accuracy: 0.8818 - val_loss: 5.8123
Epoch 4/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 1s 550us/step - accuracy: 0.8898 - loss: 4.9820 - val_accuracy: 0.8845 - val_loss: 5.6028
Epoch 5/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 1s 558us/step - accuracy: 0.8894 - loss: 4.7905 - val_accuracy: 0.8672 - val_loss: 6.2826
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 502us/step - accuracy: 0.8776 - loss: 5.9887
